# 0 - Pytorch Basics
[PyTorch](https://pytorch.org/tutorials/beginner/basics/intro.html) is an open-source machine learning library widely used for deep learning applications. It provides dynamic computation graphs and GPU acceleration, making it a preferred choice for researchers and developers.

## Importing PyTorch

Before using PyTorch, we need to import the necessary modules. 
- `torch` is the core PyTorch library.
- `torch.nn` provides neural network layers and activation functions.
- `torch.optim` includes optimizers like SGD and Adam.
- `torch.utils.data` provides utilities for handling datasets and dataloaders.



In [5]:
# import
import warnings
warnings.filterwarnings('ignore')
import torch
import torch.nn as nn
import torch.optim as optim

import pandas as pd 
import numpy as np 
import os

## Tensor
Tensors are the basic data structures in PyTorch, similar to NumPy arrays but with additional functionality, such as GPU support.

### Initializing

In [6]:
# torch.tesnor([values]): create a tensor with the given values
x = torch.tensor([1.0, 2.0, 3.0]) 
print("Tensor x: ", x)

# torch.zeros([shape]): create a tensor of zeros with the given shape
y = torch.ones(2,2) 
print("Tensor of ones:", y)

# torch.ones([shape]): create a tensor of ones with the given shape
z = torch.zeros(2,2)
print("Tensor of zeros:", z)

# torch.rand([shape]): create a tensor of random values with the given shape
rand_tensor = torch.rand(2,2)
print("Random Tensor:", rand_tensor)

# Attributes of a tensor
print("\nShape of tensor:", rand_tensor.shape)
print("Datatype of tensor:", rand_tensor.dtype)
print("Device tensor is stored on:", rand_tensor.device)

# By default, tensors are created on the CPU. We need to explicitly move tensors to the accelerator using .to method (after checking for accelerator availability).
if torch.cuda.is_available():
    tensor = torch.tensor.to('cuda')

Tensor x:  tensor([1., 2., 3.])
Tensor of ones: tensor([[1., 1.],
        [1., 1.]])
Tensor of zeros: tensor([[0., 0.],
        [0., 0.]])
Random Tensor: tensor([[0.7685, 0.0147],
        [0.1023, 0.3740]])
Shape of tensor: torch.Size([2, 2])
Datatype of tensor: torch.float32
Device tensor is stored on: cpu


### Operations on Tensors
Over 1200 tensor operations, including arithmetic, linear algebra, matrix manipulation (transposing, indexing, slicing), sampling and more are comprehensively described [here](https://pytorch.org/docs/stable/torch.html).

The torch package contains data structures for multi-dimensional tensors and defines mathematical operations over these tensors. Additionally, it provides many utilities for efficient serialization of Tensors and arbitrary types, and other useful utilities.

It has a CUDA counterpart, that enables you to run your tensor computations on an NVIDIA GPU with compute capability >= 3.0.

In [7]:
tensor = torch.ones(4, 4)

# Standard numpy-like indexing and slicing
print("\nFirst row:", tensor[0])
print("First column:", tensor[:, 0])
print("Last column:", tensor[:, -1])
print(tensor)

# Concatenating tensors
t1 = torch.cat([tensor, tensor, tensor], dim=1)
print("\nConcatenated tensor:", t1)

# Stacking tensors
t2 = torch.stack([tensor, tensor, tensor], dim=1)
print("\nStacked tensor:", t2)

# Arithmetic operations
# This computes the matrix multiplication between two tensors. y1, y2, y3 will have the same value
# .T returns the transpose of a tensor
y1 = tensor @ tensor.T
y2 = tensor.matmul(tensor.T)

y3 = torch.rand_like(y1)
torch.matmul(tensor, tensor.T, out=y3)

# This computes the element-wise product. z1, z2, z3 will have the same value
z1 = tensor * tensor
z2 = tensor.mul(tensor)

z3 = torch.rand_like(tensor)
torch.mul(tensor, tensor, out=z3)



tensor([1, 2, 3, 4, 5, 1, 2, 3, 4, 5])

## DataLoader
DataLoaders are essential for efficiently handling large datasets, especially when training deep learning models. They provide:
- **Batching**: Splitting the dataset into small batches for efficient training.
- **Shuffling**: Randomizing the order of data to prevent patterns from influencing training.
- **Parallel loading**: Loading data in parallel using multiple CPU threads.

In [8]:
from torch.utils.data import Dataset
from ucimlrepo import fetch_ucirepo

data = fetch_ucirepo(id=53)
x = data['data']['features']
y = data['data']['targets']

print("Dataset: ", data['metadata']['name'])
print("Dim: ", x.shape)
print("Targets: ", y['class'].unique())

x.head()

Dataset:  Iris
Dim:  (150, 4)
Targets:  ['Iris-setosa' 'Iris-versicolor' 'Iris-virginica']


,sepal length,sepal width,petal length,petal width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [9]:
from sklearn.preprocessing import LabelEncoder

# encode the target variable
label_encoder = LabelEncoder()
y['class'] = label_encoder.fit_transform(y)

# check if the number of rows in x is equal to the number of targets
assert(x.shape[0] == y.shape[0])

y.head()

,class
0,0
1,0
2,0
3,0
4,0


Since we are using Pytorch, we might want to convert the dataset we are using to a Pytorch Dataset. A custom Dataset class must implement three functions: init, len, and getitem.

In [10]:
class CustomDataset(Dataset):
    def __init__(self, targets_file, data_file, transform=None, target_transform=None):
        self.targets_file = pd.read_csv(targets_file)
        self.data_dir = pd.read_csv(data_file)
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.targets_file)

    def __getitem__(self, idx):
        return self.data_dir.iloc[idx].to_numpy(dtype=np.float32), self.targets_file.iloc[idx].item()

# save the custom dataset
folder = os.path.join('data', data['metadata']['name'])
os.makedirs(folder, exist_ok=True)
x.to_csv(os.path.join(folder, 'data.csv'), index=False)
y.to_csv(os.path.join(folder, 'targets.csv'), index=False)

In [11]:
dataset = CustomDataset(targets_file=os.path.join(folder, 'targets.csv'), data_file=os.path.join(folder, 'data.csv'))

from torch.utils.data import DataLoader

# split the dataset into train and test
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# Now we can iterate over the train and test datasets.
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True)